In [15]:
import random

In [26]:
import cv2
import os
import numpy as np
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Define categories and initialize data array
categories = ["WithMask", "WithoutMask"]
data = []

# Load and preprocess images
for category in categories:
    path = os.path.join("Train", category)
    label = categories.index(category)
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        data.append([img, label])

# Shuffle data to mix 'WithMask' and 'WithoutMask' images
np.random.shuffle(data)

# Separate features (X) and labels (y)
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Normalize the pixel values
X = X / 255.0

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the VGG16 model with pretrained weights, excluding the top layers
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.trainable = False

# Create a new model using VGG16 as the base
model = Sequential([
    vgg,
    GlobalAveragePooling2D(),  # Pool the 7x7x512 output to 512
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Function to detect face mask
def detect_face_mask(img):
    img = cv2.resize(img, (224, 224))  # Resize image to 224x224 if not already
    img = img / 255.0  # Normalize the image
    y_pred = model.predict(np.expand_dims(img, axis=0))  # Predict
    return "With Mask" if y_pred[0] > 0.5 else "Without Mask"



Epoch 1/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 527s 9s/step - accuracy: 0.7637 - loss: 0.5035 - val_accuracy: 0.9267 - val_loss: 0.2643
Epoch 2/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 499s 9s/step - accuracy: 0.9325 - loss: 0.2175 - val_accuracy: 0.9378 - val_loss: 0.1813
Epoch 3/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 504s 9s/step - accuracy: 0.9497 - loss: 0.1566 - val_accuracy: 0.9467 - val_loss: 0.1620
Epoch 4/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 508s 9s/step - accuracy: 0.9634 - loss: 0.1159 - val_accuracy: 0.9578 - val_loss: 0.1303
Epoch 5/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 515s 9s/step - accuracy: 0.9703 - loss: 0.0958 - val_accuracy: 0.9467 - val_loss: 0.1277
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step
Without Mask


In [38]:
def detect_face_mask(img):
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)
    
    y_pred = model.predict(img_input)
    print("Prediction Probability: ", y_pred[0])
    
    # Adjusted threshold for classification
    return 0 if y_pred[0] < 0.4 else 1  # Experiment with this threshold

# Test with a sample image
sample1 = cv2.imread('ju.jpeg')
result = detect_face_mask(sample1)

if result == 0:
    print("Mask Detected")
else:
    print("No Mask Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
Prediction Probability:  [0.49647132]
No Mask Detected


In [ ]:
import cv2
import numpy as np

# Function to draw label on the frame
def draw_label(frame, label, pos, bg_color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.8
    thickness = 2
    text_size = cv2.getTextSize(label, font, scale, thickness)[0]
    end_x = pos[0] + text_size[0] + 2
    end_y = pos[1] - text_size[1] - 2
    cv2.rectangle(frame, pos, (end_x, end_y), bg_color, -1)
    cv2.putText(frame, label, pos, font, scale, (255, 255, 255), thickness)

# Load the pre-trained face detection model (Haar cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # If a face is detected, run mask detection on the face region
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_img_resized = cv2.resize(face_img, (224, 224))
            y_pred = detect_face_mask(face_img_resized)
            
            # Draw label on the frame
            if y_pred == 0:
                draw_label(frame, "Mask", (x, y-10), (0, 255, 0))
            else:
                draw_label(frame, "No Mask", (x, y-10), (0, 0, 255))
    else:
        # If no face is detected, show "No Face Detected" label
        draw_label(frame, "No Face Detected", (30, 30), (0, 0, 255))
    
    # Show the frame in a window
    cv2.imshow("window", frame)
    
    # Break the loop when 'x' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
